In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from pprint import pprint
import json
import random
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from tqdm import trange, tqdm

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
TEST_X_AUX_PATH = '/Users/reza/School/2025/1-Spring/Advanced ML/Project/data/new/preprocessed_selected_features/test/aux.csv'
TEST_X_FNC_PATH = '/Users/reza/School/2025/1-Spring/Advanced ML/Project/data/new/preprocessed_selected_features/test/connectome_matrices.csv'

In [18]:
test_X_fnc_df = pd.read_csv(TEST_X_FNC_PATH)
test_X_aux_df = pd.read_csv(TEST_X_AUX_PATH)
test_X_fnc_df.set_index('participant_id', inplace=True)
test_X_aux_df.set_index('participant_id', inplace=True)
test_X_aux_df = test_X_aux_df.reindex(test_X_fnc_df.index)

In [19]:
test_X_fnc_df.head()

,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,0throw_10thcolumn,...,195throw_196thcolumn,195throw_197thcolumn,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn
participant_id,,,,,,,,,,,,,,,,,,,,,
Cfwaf5FX7jWK,0.665269,0.786008,0.629912,0.571596,0.756816,0.834951,0.785671,0.439002,0.603705,0.535544,...,0.109113,-0.080147,-0.112746,-0.035289,0.500307,0.404156,-0.015204,0.728375,0.623918,0.615181
vhGrzmvA3Hjq,0.518820,0.398501,0.455240,0.395056,0.582186,0.370295,0.442388,0.341152,0.443344,0.312906,...,0.268648,-0.001063,0.106298,0.040809,0.773208,0.353462,0.865709,0.505250,0.622880,0.387470
ULliyEXjy4OV,0.169292,0.428437,-0.098085,0.213280,0.091299,0.018532,0.412206,0.076936,0.415742,0.343500,...,0.069627,0.034699,-0.072661,0.144986,0.647908,0.597701,0.635925,0.661952,0.678780,0.831323
LZfeAb1xMtql,0.162000,0.857543,0.470499,0.514128,0.619466,0.685745,0.542278,0.388654,0.515506,0.475184,...,0.062658,-0.350891,0.155848,-0.322303,0.727458,0.686660,0.733794,0.651472,0.760400,0.600308
EnFOUv0YK1RG,0.153677,0.633297,0.575747,0.395263,0.464511,0.484715,0.488523,0.251631,0.328348,0.346419,...,0.428369,0.003280,0.371870,0.425030,0.540083,0.623086,0.539733,0.320658,0.165316,0.597570


In [20]:
test_X_aux_df.head()

,Basic_Demos_Enroll_Year_2016,PreInt_Demos_Fam_Child_Ethnicity_2.0,Barratt_Barratt_P2_Occ_25.0,SDQ_SDQ_Prosocial,PreInt_Demos_Fam_Child_Race_4.0,PreInt_Demos_Fam_Child_Ethnicity_1.0,Barratt_Barratt_P2_Occ_20.0,Basic_Demos_Enroll_Year_2017,Barratt_Barratt_P1_Edu_18.0,Barratt_Barratt_P2_Edu_15.0,...,Barratt_Barratt_P2_Occ_10.0,SDQ_SDQ_Emotional_Problems,ColorVision_CV_Score,APQ_P_APQ_P_OPD,PreInt_Demos_Fam_Child_Race_8.0,Barratt_Barratt_P1_Edu_9.0,Barratt_Barratt_P2_Occ_15.0,Basic_Demos_Enroll_Year_2019,APQ_P_APQ_P_PP,Basic_Demos_Study_Site_3
participant_id,,,,,,,,,,,,,,,,,,,,,
Cfwaf5FX7jWK,0,0,0,0.8,0,0,0,0,0,0,...,0,0.3,1.000000,0.571429,0,0,0,0,0.80,0
vhGrzmvA3Hjq,0,0,0,0.9,0,0,0,0,0,0,...,0,0.8,0.857143,0.523810,0,0,0,0,0.90,0
ULliyEXjy4OV,0,0,0,0.9,0,0,0,0,0,0,...,0,0.1,0.928571,0.428571,0,0,0,0,0.75,0
LZfeAb1xMtql,0,0,0,0.6,0,0,0,0,0,0,...,0,0.4,0.928571,0.476190,0,0,0,0,0.85,0
EnFOUv0YK1RG,0,1,0,1.0,0,0,0,0,1,0,...,0,0.6,1.000000,0.571429,0,0,0,0,0.90,0


In [21]:
class Model(nn.Module):
    def __init__(self, input_dim, layer_dims, dropout=0.5, output_dim=4):
        super(Model, self).__init__()
        layers = []
        prev_dim = input_dim
        for dim in layer_dims:
            layers.append(nn.Linear(prev_dim, dim))
            layers.append(nn.ReLU())
            if dropout > 0:
                layers.append(nn.Dropout(dropout))
            prev_dim = dim
        layers.append(nn.Linear(prev_dim, output_dim))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

In [22]:
X_fnc = np.array(test_X_fnc_df.values, dtype=np.float32)
X_aux = np.array(test_X_aux_df.values, dtype=np.float32)
X_fnc.shape, X_aux.shape

((304, 19900), (304, 59))

In [26]:
fnc_model = Model(
    input_dim=X_fnc.shape[1], layer_dims=[128, 64, 32, 16], dropout=0.3
).to(device)
state_dict_fnc = torch.load('/Users/reza/School/2025/1-Spring/Advanced ML/Project/WiDS2025/results/unimodal/mlp_fnc/128-64-32-16-0.3.pt', map_location=device)
fnc_model.load_state_dict(state_dict_fnc)
fnc_model.eval()

/var/folders/bv/7h0f0hns2y72yqgg3ww8qkkm0000gn/T/ipykernel_8765/3374767233.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict_fnc = torch.load('/Users/reza/Schoo

Model(
  (layers): Sequential(
    (0): Linear(in_features=19900, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=32, out_features=16, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.3, inplace=False)
    (12): Linear(in_features=16, out_features=4, bias=True)
  )
)

In [27]:
aux_model = Model(
    input_dim=X_aux.shape[1], layer_dims=[128, 64, 32, 16], dropout=0.3
).to(device)
state_dict_fnc = torch.load('/Users/reza/School/2025/1-Spring/Advanced ML/Project/WiDS2025/results/unimodal/mlp_aux/128-64-32-16-0.3.pt', map_location=device)
aux_model.load_state_dict(state_dict_fnc)
aux_model.eval()

/var/folders/bv/7h0f0hns2y72yqgg3ww8qkkm0000gn/T/ipykernel_8765/707389310.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict_fnc = torch.load('/Users/reza/School

Model(
  (layers): Sequential(
    (0): Linear(in_features=59, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=32, out_features=16, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.3, inplace=False)
    (12): Linear(in_features=16, out_features=4, bias=True)
  )
)

In [28]:
seed = 42  # Choose any fixed number
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # If using CUDA

In [29]:
out_fnc = fnc_model(torch.from_numpy(X_fnc).to(device)).cpu().detach().numpy()

In [30]:
out_fnc, out_fnc.shape

(array([[-0.16359964, -0.74035966,  0.48728377,  0.21424264],
        [-0.3813761 , -0.38307422,  0.31557837,  0.29766938],
        [-0.44715047,  0.13851526, -0.04981585,  0.29829618],
        ...,
        [ 0.09677386, -0.39240837,  0.20618835,  0.07971245],
        [-0.44027185,  0.24748856,  0.0120559 ,  0.11848335],
        [-0.310422  , -0.02427318, -0.00453328,  0.30538383]],
       dtype=float32),
 (304, 4))

In [31]:
out_aux = aux_model(torch.from_numpy(X_aux).to(device)).cpu().detach().numpy()

In [32]:
out_aux, out_aux.shape

(array([[-0.27041447, -0.30498773,  0.44786397,  0.56787366],
        [ 0.06906556,  0.70095664, -0.62716794, -0.30161616],
        [ 0.29695088,  0.44952077, -0.27300557, -0.2873833 ],
        ...,
        [ 0.8827038 ,  1.022814  , -0.7447747 , -1.0439894 ],
        [-0.2442146 , -0.44020814,  0.6244849 ,  0.6457081 ],
        [ 0.95661175,  0.643394  , -0.33019516, -0.8502949 ]],
       dtype=float32),
 (304, 4))

In [33]:
pred_fnc = np.argmax(out_fnc, axis=1)
np.unique(pred_fnc, return_counts=True)

(array([0, 1, 2, 3]), array([ 71,  63, 118,  52]))

In [34]:
pred_aux = np.argmax(out_aux, axis=1)
np.unique(pred_aux, return_counts=True)

(array([0, 1, 2, 3]), array([ 18, 108,  31, 147]))

In [35]:
out_avg = np.mean([out_fnc, out_aux], axis=0)
out_avg.shape
pred = np.argmax(out_avg, axis=1)

In [37]:
mapping = {
    0: (0, 0),  # Not ADHD, Male
    1: (1, 0),  # ADHD, Male
    2: (0, 1),  # Not ADHD, Female
    3: (1, 1)   # ADHD, Female
}

# Convert array to DataFrame
df = pd.DataFrame([mapping[val] for val in pred], columns=['ADHD_Outcome', 'Sex_F'])

# Assign participant IDs (e.g., from test_X indices)
df['participant_id'] = test_X_fnc_df.index.values

# Optional: set participant_id as the index
df.set_index('participant_id', inplace=True)

print(df)
df.to_csv('bimodal_mlp_average.csv')

                ADHD_Outcome  Sex_F
participant_id                     
Cfwaf5FX7jWK               0      1
vhGrzmvA3Hjq               1      0
ULliyEXjy4OV               1      0
LZfeAb1xMtql               1      1
EnFOUv0YK1RG               0      1
...                      ...    ...
UadZfjdEg7eG               1      1
IUEHiLmQAqCi               0      1
cRySmCadYFRO               0      0
E3MvDUtJadc5               1      1
dQJXfyRazknD               0      0

[304 rows x 2 columns]
